In [2]:
import timeit
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Create embeddings. 

In [3]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [4]:
df = pd.read_csv('Indicator-Desc_Data.csv')

In [5]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel

/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
#model = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # PyTorch
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow


2023-05-09 10:28:21.994788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-ca

In [7]:
# Initialize an empty array to store the embeddings
embeddings = np.zeros((len(df), 768))

# Loop through the dataframe column and generate embeddings
for i, text in enumerate(df['content']):
    # Encode the text using the tokenizer
    input_ids = tokenizer.encode(str(text), add_special_tokens=True, return_tensors="tf")
    # Generate the embeddings using the model
    output = model(input_ids)
    # Extract the embeddings from the output and flatten using max-pooling
    embeddings[i] = np.max(output.last_hidden_state.numpy(), axis=1)
    
# Add the embeddings to the dataframe
for j in range(768):
    df[f'embedding_{j}'] = embeddings[:, j]


/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_91672/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{j}'] = embeddings[:, j]
/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_91672/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{j}'] = embeddings[:, j]
/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_91672/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [8]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [9]:
df.to_csv('transformers_data.csv', index=False)

## XGBOOST

In [6]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [17]:
#df = pd.read_csv('transformers_data.csv')
df = pd.read_csv('transformers_data_High-RC.csv')

In [13]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBRegressor



In [18]:
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
# create XGBRegressor model with default hyperparameters
model = XGBRegressor(max_depth=3, learning_rate=0.1)

# train the model on your training data (X_train and y_train)
model.fit(X_train, y_train)

# use the trained model to make predictions on your test data (X_test)
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error, r2_score

# Compute the MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
rsq = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", rsq)

Mean Squared Error: 0.2367430060073809
R-squared: 0.09838400670173764


Results of XGBoost for transformers_data.csv:
MSE: 0.23128397706649886
R-squared: 0.11777291394938094

Results of XGBoost for transformers-high RC:
MSE: 0.2367430060073809
R-squared: 0.09838400670173764

## CATBOOST

In [27]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBRegressor

In [28]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [29]:
df = pd.read_csv('transformers_data.csv')

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function='RMSE',
    verbose=False
)

# Train the model using the training set, and evaluate it using the testing set
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=False
)

# Get the predictions on the testing set
y_pred = model.predict(X_test)

# Compute the MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
rsq = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", rsq)


In [ ]:
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from catboost import CatBoostRegressor

# Create the model
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6)

# Train the model
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

# Make predictions
y_pred = catboost_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

## Random Forest

In [24]:
df = pd.read_csv('transformers_data_High-RC.csv')

In [25]:
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


# Create a Random Forest Regressor with 100 trees
rf = RandomForestRegressor(n_estimators=200, random_state=42)

# Train the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Compute the MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
rsq = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", rsq)

Mean Squared Error: 0.23578732114591858
R-squared: 0.10202365278961656


Results of Random Forest for transformers_data.csv:
MSE: 0.23209107807828505
R-squared: 0.11469424683716722

Results of Random Forest for transformers-high RC:
MSE: 0.23578732114591858
R-squared: 0.10202365278961656